<a href="https://colab.research.google.com/github/kapeeshvarma/Play-Store-App-Analysis/blob/main/Play_Store_App_Review_Analysis_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Play Store App Review Analysis**

Over the years, Google Play Store has evolved into a one-stop platform for downloading apps from different categories. It is the central hub for millions of apps for your smartphone. In this project, we will do a comprehensive analysis of the Play Store App by comparing over ten thousand apps and reviews across different categories. We'll look for insights in the data to devise strategies to drive growth and retention.

The data used for this project consists of two files:


*   Play Store Data.csv : This dataset contains all the details of the applications on Google Play Store. There are 13 features for each app.
*   User Reviews.csv : It contains reviews for each app along with Sentiment (Positive, Negative or Neutral), Sentiment Polarity and Sentiment Subjectivity.



##Importing Data and Modules

In [50]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
# Import the necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [52]:
# Read in the dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Play Store App Review Analysis/Play Store Data.csv')

# Print the summary of the dataframe
print(data.info())

# Print the total number of apps
print('Total number of apps in the dataset = ', len(data))

# Have a look at a random sample of the dataframe
data.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB
None
Total number of apps in the dataset =  10841


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
7204,TI-84 CE Graphing Calculator Manual TI 84,FAMILY,5.0,1,27M,100+,Paid,$4.99,Everyone,Education,"March 28, 2018",1.5.2,4.1 and up
729,ABC Preschool Free,EDUCATION,3.8,27572,25M,"5,000,000+",Free,0,Everyone,Education;Education,"October 25, 2017",3.0,2.3 and up
6316,BJ Bridge Acol Beginner 2018,GAME,3.7,11,4.8M,"5,000+",Free,0,Everyone,Card,"May 21, 2018",6.2-begina,4.0 and up
7257,Wallpapers DAF CF 85 Trucks,PERSONALIZATION,NaN,0,16M,10+,Free,0,Teen,Personalization,"April 29, 2017",1.0,4.1 and up
10628,PriorityONE Credit Union of Fl,FINANCE,4.7,178,12M,"1,000+",Free,0,Everyone,Finance,"June 12, 2018",5.9.1.0,5.0 and up
3384,ASUS Cover for ZenFone 2,PERSONALIZATION,4.4,43960,9.8M,"10,000,000+",Free,0,Everyone,Personalization,"December 1, 2017",2.0.0.39_171124,5.0 and up
3790,USA TODAY,NEWS_AND_MAGAZINES,4.1,49259,Varies with device,"5,000,000+",Free,0,Everyone 10+,News & Magazines,"August 2, 2018",Varies with device,Varies with device
6692,cronometra-br,PRODUCTIVITY,NaN,0,5.4M,0+,Paid,$154.99,Everyone,Productivity,"November 24, 2017",1.0.0,4.1 and up
7064,BZ Langenthaler Tagblatt,VIDEO_PLAYERS,NaN,22,8.5M,"1,000+",Free,0,Everyone,Video Players & Editors,"August 3, 2018",7.5.9.1,4.1 and up
10563,FK Spartak Subotica,SPORTS,NaN,0,26M,10+,Free,0,Everyone,Sports,"February 19, 2018",1.0,4.1 and up
